In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch_geometric.nn as gnn
import pickle
from tqdm import tqdm
import numpy as np
from torch.functional import F
import matplotlib.pyplot as plt


with open("data/train_graphs_data.pkl", "rb") as file:
    all_graphs = pickle.load(file)
    file.close()

In [16]:
len(all_graphs), all_graphs[0], all_graphs[0].y[0]

(90,
 Data(x=[1140, 33], edge_index=[2, 11709], edge_attr=[11709, 2], y=[1140, 2]),
 tensor([-0.2741, -0.6940]))

# 制作模型

### SAGE用于对图进行embedding

In [17]:
class SAGEModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SAGEModel, self).__init__()

        self.graph_features = nn.ModuleList(
            [
                gnn.SAGEConv(input_size, 128),
                nn.LayerNorm(128),
                nn.ReLU(),
                gnn.SAGEConv(128, 128),
                nn.LayerNorm(128),
                nn.ReLU(),
                gnn.SAGEConv(128, 128),
                nn.LayerNorm(128),
                nn.ReLU(),
            ]
        )

        # 考虑更多的trick，如layernorm等等
        self.regression = nn.Sequential(
            nn.Linear(128, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.LayerNorm(32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, output_size),
        )

    def forward(self, data):
        x, edge_index, _ = data.x, data.edge_index, data.edge_attr

        output = x

        for layer in self.graph_features:
            if isinstance(layer, gnn.SAGEConv):
                output = layer(output, edge_index)
            else:
                output = layer(output)

        output = self.regression(output)

        return output

In [18]:
# 看看模型的输入输出
model = SAGEModel(input_size=33, output_size=32)
output = model(all_graphs[0])
output.shape

torch.Size([1140, 32])

### LSTM用于串联每个节点的embedding，并计算回归值

输入：x.shape = (1140, seq_len, 节点feature_size)

**注意：第一个维度只能是1140，而不能自定义batch_size，因为图模型跑完以后会得到整张图的embedding**

如果选取一小部分放进lstm，SAGE+LSTM模型的forward函数将变得很难写

In [19]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LSTMModel, self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.features = nn.Sequential(
            nn.LSTM(
                input_size=input_size,
                hidden_size=64,
                num_layers=1,
                batch_first=True,
            ),
        )

        self.linears = nn.Sequential(
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, output_size),
        )

    def forward(self, x):
        out, (_, _) = self.features(x)
        out = self.linears(out)

        return out

In [20]:
lstm_input = np.array([g.x for g in all_graphs])
# batch放到第一个维度
lstm_input = lstm_input.swapaxes(0, 1)
lstm_input = torch.from_numpy(lstm_input).float()

lstm_input.shape

torch.Size([1140, 90, 33])

In [21]:
model = LSTMModel(input_size=33, output_size=2)
output = model(lstm_input)
output.shape

torch.Size([1140, 90, 2])

## 总的模型（下面简称为SL模型）：先图conv，再lstm

输入：图列表，长度为LSTM的seq_len

输出：[1140, seq_len, 2]

In [22]:
class SAGEandLSTMModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SAGEandLSTMModel, self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.sage = SAGEModel(input_size, 32)
        self.lstm = LSTMModel(32, output_size)

    def forward(self, graphs):
        # data: [graph]
        # 堆叠在维度1
        # output.shape = [1140个节点, 90张图, 32个特征]

        # 这里能不能并行呢？能不能不用for循环？
        output = torch.stack([self.sage(graph) for graph in graphs], dim=1)
        output = self.lstm(output)

        return output

In [23]:
model = SAGEandLSTMModel(input_size=33, output_size=2)
output = model(all_graphs)

output.shape

torch.Size([1140, 90, 2])

# 制作数据

数据需要处理成SL模型需要的格式

In [24]:
# 数据全部放到GPU上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
graphs = [graph.to(device) for graph in all_graphs]

In [25]:
def genLSTMData(graphs, seq_len):
    # graphs：列表，每个元素是一个graph
    graph_list = []
    for i in range(len(graphs) - seq_len + 1):
        graph_list.append(graphs[i : i + seq_len])
    return graph_list

In [26]:
graph_list = genLSTMData(graphs, 14)

### 划分训练测试集

In [27]:
# 划分训练集和验证集，使用sklearn的train_test_split函数
from sklearn.model_selection import train_test_split

train_graph_list, val_graph_list = train_test_split(graph_list, test_size=0.1)

# 开始训练

In [28]:
input_size = graphs[0].x.shape[1]
# 这里的output_size就是最终的输出维度，不需要再乘以num_heads
output_size = graphs[0].y.shape[1]
# 个人认为，注意力头的个数至少应该等于输出维度，因为每个输出可能需要关注不同的邻居
num_heads = graphs[0].y.shape[1]

# batch_size = 256
input_size, output_size, num_heads, device

(33, 2, 2, device(type='cuda'))

In [29]:
def train(
    model, criterion, optimizer, train_graph_list, val_graph_list=None, num_epochs=50
):
    # 训练过程记录
    train_loss_list = []
    val_loss_list = []

    with tqdm(total=num_epochs, desc="Training Progress", unit="epoch") as pbar_epochs:
        for epoch in range(num_epochs):
            # 训练
            model.train()
            train_loss = 0.0

            # 图网络不支持batch，只能一个一个图训练
            for i, graphs in enumerate(train_graph_list):
                output = model(graphs)

                optimizer.zero_grad()
                loss = criterion(
                    output, torch.stack([graph.y for graph in graphs], dim=1)
                )
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

            train_loss /= len(train_graph_list)
            train_loss_list.append(train_loss)

            # 验证
            if val_graph_list:
                model.eval()
                with torch.no_grad():
                    val_loss = 0.0
                    for i, graphs in enumerate(val_graph_list):
                        output = model(graphs)
                        loss = criterion(
                            output, torch.stack([graph.y for graph in graphs], dim=1)
                        )
                        val_loss += loss.item()
                    val_loss /= len(val_graph_list)
                    val_loss_list.append(val_loss)

            if val_graph_list:
                pbar_epochs.set_postfix(
                    {"train MSE Loss": train_loss, "val MSE Loss": val_loss}
                )
            else:
                pbar_epochs.set_postfix({"train MSE Loss": train_loss})
            pbar_epochs.update(1)

    # 可视化训练过程
    plt.figure()
    plt.plot(train_loss_list, label="train loss")
    if val_graph_list:
        plt.plot(val_loss_list, label="val loss")
    plt.xlabel("epoch")
    plt.ylabel("MSE loss")
    plt.legend()

    return model

In [30]:
# 定义模型、算法、损失函数
model = SAGEandLSTMModel(
    input_size=input_size,
    output_size=output_size,
).to(device)
# 考虑是否加入weight_decay
optimizer = optim.Adam(model.parameters(), lr=1e-4)
# criterion = nn.MSELoss(reduction="sum")
criterion = nn.MSELoss().to(device)

model = train(
    model, criterion, optimizer, train_graph_list, val_graph_list, num_epochs=300
)

Training Progress:  39%|███▉      | 117/300 [07:43<12:04,  3.96s/epoch, train MSE Loss=0.0118, val MSE Loss=0.00931]


KeyboardInterrupt: 

### 把所有数据都放进模型训练

In [ ]:
# # 定义模型、算法、损失函数
# model = GATv2Model(
#     input_size=input_size,
#     output_size=output_size,
#     num_heads=num_heads,
# ).to(device)
# # 考虑是否加入weight_decay
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# criterion = nn.MSELoss()

# model = train(model, criterion, optimizer, graphs, num_epochs=300)

In [ ]:
# 保存模型
model_path = "saved/models/sage+lstm.pth"
torch.save(model.state_dict(), model_path)

# 预测

In [ ]:
# 读取模型
model = GATv2Model(
    input_size=input_size,
    output_size=output_size,
    num_heads=num_heads,
).to(device)
model.load_state_dict(torch.load(model_path))

In [ ]:
# 读取node_ids
with open("data/node_ids.pkl", "rb") as file:
    node_ids = pickle.load(file)
    file.close()

In [ ]:
# 读取scaler
import pickle

with open("saved/scaler/label_scaler.pkl", "rb") as file:
    label_scaler = pickle.load(file)
    file.close()

In [ ]:
# 读取测试集
with open("data/test_graphs_data.pkl", "rb") as file:
    test_graphs = pickle.load(file)
    file.close()

In [ ]:
len(test_graphs), test_graphs[0]

预测

In [ ]:
model.eval()
output_list = []
with torch.no_grad():
    for i, graph in enumerate(test_graphs):
        graph = graph.to(device)
        output = model(graph)
        output_list.append(output.detach().cpu().numpy())

In [ ]:
len(output_list), output_list[0].shape, output_list[0][0]

In [ ]:
# 缩放
output_list = [label_scaler.inverse_transform(output) for output in output_list]

In [ ]:
# 看看是不是缩放完成啦
output_list[0][0]

### 把输出转成需要的格式

In [ ]:
import csv

# 结果写入csv，分隔符为\t
output_path = "output/gatv2_out.csv"

date_id = [20230404, 20230405, 20230406, 20230407]
with open(output_path, "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")  # 设置分隔符为制表符
    writer.writerow(["geohash_id", "consumption_level", "activity_level", "date_id"])

    # output_list.shape = [4天, 节点数, 2个输出]
    for nidx in range(len(node_ids)):
        for day in range(len(date_id)):
            # 注意不要写反了
            writer.writerow(
                [
                    node_ids[nidx],
                    output_list[day][nidx][1],
                    output_list[day][nidx][0],
                    date_id[day],
                ]
            )
    f.close()